In [13]:
import pandas as pd 
import base64
import pyModeS as pms
df = pd.read_csv("test.csv",sep = ";")
df = df.drop(columns="Unnamed: 2")
df

,ts_kafka,message
0,1734667201534,oAANuYAVzS4ABLg3aB0=
1,1734667201534,kDQjSSgABjG4hW4fTbY=
2,1734667201534,XTxUM/+DTw==
3,1734667201534,kDQjShBRgxXDaCAWeDM=
4,1734667201534,jTRV1fgjAAIASbigrIs=
...,...,...
8354,1734667796655,IAAAALRFCw==
8355,1734667796655,IAAAALRFCw==
8356,1734667796655,jTlkQ5kItjeQCBj8Xms=
8357,1734667796655,IAAGM27mPA==


In [14]:
def encodeHex(b64):
    return base64.b64decode(b64).hex()

In [18]:
def getDownlink(hex):
    return pms.df(hex)

In [21]:
def msgIsCorrupted(hex):
    return (pms.crc(hex) != 0)

In [24]:
def getICAO(hex):
    return str(pms.common.icao(hex))

In [39]:
def getOnGround(hex):
    decimal_value =  pms.bin2int(pms.hex2bin(hex)[5:8]) 
    if decimal_value == 4:
        return 1
    elif decimal_value == 5:
        return 0
    else:
        return None
    

In [29]:
df["messageHex"] = df["message"].apply(encodeHex)
df["DL"] = df["messageHex"].apply(getDownlink)
df_11 = df[df["DL"] == 11]
df_11 =df_11.reset_index()


In [30]:
df_11

,index,ts_kafka,message,messageHex,DL
0,2,1734667201534,XTxUM/+DTw==,5d3c5433ff834f,11
1,13,1734667201534,XavTNsUSrg==,5dabd336c512ae,11
2,14,1734667201534,XavTNsUSrg==,5dabd336c512ae,11
3,32,1734667201534,XjQjVJiA3Q==,5e3423549880dd,11
4,79,1734667206276,XTSBC8ML9Q==,5d34810bc30bf5,11
...,...,...,...,...,...
661,8309,1734667795441,XTRV1ZpM+g==,5d3455d59a4cfa,11
662,8310,1734667795441,XTRV1ZpM+g==,5d3455d59a4cfa,11
663,8323,1734667796146,XTR1TaOehw==,5d34754da39e87,11
664,8324,1734667796146,XjQjVWd01A==,5e3423556774d4,11


In [31]:
df_11["ICAO"] = df_11["messageHex"].apply(getICAO)

In [ ]:
df_11['timestamp'] = pd.to_datetime(df_11['ts_kafka'], unit='ms')
df_11['formatted_date'] = df_11['timestamp'].dt.strftime('%d/%m/%Y %H:%M:%S')



,index,ts_kafka,message,messageHex,DL,ICAO,timestamp,formatted_date
0,2,1734667201534,XTxUM/+DTw==,5d3c5433ff834f,11,3c5433,2024-12-20 04:00:01.534,20/12/2024 04:00:01
1,13,1734667201534,XavTNsUSrg==,5dabd336c512ae,11,abd336,2024-12-20 04:00:01.534,20/12/2024 04:00:01
2,14,1734667201534,XavTNsUSrg==,5dabd336c512ae,11,abd336,2024-12-20 04:00:01.534,20/12/2024 04:00:01
3,32,1734667201534,XjQjVJiA3Q==,5e3423549880dd,11,342354,2024-12-20 04:00:01.534,20/12/2024 04:00:01
4,79,1734667206276,XTSBC8ML9Q==,5d34810bc30bf5,11,34810b,2024-12-20 04:00:06.276,20/12/2024 04:00:06
...,...,...,...,...,...,...,...,...
661,8309,1734667795441,XTRV1ZpM+g==,5d3455d59a4cfa,11,3455d5,2024-12-20 04:09:55.441,20/12/2024 04:09:55
662,8310,1734667795441,XTRV1ZpM+g==,5d3455d59a4cfa,11,3455d5,2024-12-20 04:09:55.441,20/12/2024 04:09:55
663,8323,1734667796146,XTR1TaOehw==,5d34754da39e87,11,34754d,2024-12-20 04:09:56.146,20/12/2024 04:09:56
664,8324,1734667796146,XjQjVWd01A==,5e3423556774d4,11,342355,2024-12-20 04:09:56.146,20/12/2024 04:09:56


In [42]:
df_11["OnGround"] = df_11["messageHex"].apply(getOnGround)
df_11["OnGround"].isna()

0      False
1      False
2      False
3       True
4      False
       ...  
661    False
662    False
663    False
664     True
665    False
Name: OnGround, Length: 666, dtype: bool